# Data preprocessing  

The following objects are required for data preparation in sentiment analysis task when using *torchtext.data*:

- **Field**: specifies how to preprocess each data column in our dataset.
- **LabelField**: defines the label in the classification task.

The *main()* function in *a3main.py* script defines our *Field* and *LabelField* objects. We have defined the *Field* object to convert strings to lower case by passing *lower=True* argument. We have also set *include_length=True* to allow for dynamic padding by adding the lengths of the reviews to the dataset. In addition to the *Field* object, a *preprocessing()* function has been defined in *student.py* to perform the following additional preprocessing of the data using the regular expression package, *re*:

- remove html mark tags
- remove non-ascii and digits
- remove unwanted characters
- remove extra white spaces.

The *preprocessing()* function will be called after tokenising but prior to numericalising to perform the text cleaning task.

The model uses Global Vectors for Word Representation (GloVe 6B) unsupervised learning algorithm for obtaining vector representation for words (Pennington et al. 2014). GloVe is used for word embedding for text where it allows for words with similar meaning to have similar representation. The dimension of the vector was chosen to be 300 as the increase in dimension allows the vector to capture more information. For this task, dimension of 300 was found to perform better when we experimented with different dimension values (50, 150, 200, 250). However, it should be noted that increases in dimension size will result in greater computational complexity. 

# Prediction output conversion 

We must ensure the output of our network is in the same Tensor data type (LongTensor) of the *rating* and *businessCategory* of *dataset* as defined by *main()* function in *a3main.py*. This has been achieved by defining *convertNetOutput(ratingOutput, categoryOutput)* function to process the prediction label data. 

Our model uses sigmoid activation function which outputs *FloatTensor* data type values between 0 and 1 to predict ratings. In this sentiment analysis task, our model is required to predict whether the review texts are negative or positive. Therefore, the ratings prediction is a binary classification task where it only requires to take on the value of 0 (negative) or 1 (positive). We can thus ensure the prediction label for *ratingOutput* is in *LongTensor* data type by applying the following process:

- *ratingOutput = torch.tensor([1 if x > 0.5 else 0 for x in ratingOutput]).to(device)*
- prediction values over 0.5 given by our model will be assessed as positive and assigned 1 (integer64/long data type)
- prediction values below 0.5 given by our model will be assessed as negative and assigned 0 (integer64/long data type).

The business categories target class labels in our dataset are [0, 1, 2, 3, 4], which represent restaurants, shopping, home services, health and medical, and automotive respectively. Our model uses the *CrossEntropyLoss()* loss function, which combines log softmax loss function and negative log-likelihood and outputs a probability distribution between 0 and 1. Therefore, the predicted label with the highest probability was assigned to the target class by applying the following conversion process:

- *categoryOutput = tnn.Softmax(categoryOutput, dim=1)* 
- *categoryOutput = torch.argmax(categoryOutput, dim=1)*.

Prior to assigning a class label to the model's prediction *tnn.Softmax()* function is applied to to ensure the values lie in the range [0, 1] which can be interpreted as a probability distribution. Subsequently, the *torch.argmax()* function will return the indices of the maximum values of the category prediction output tensor in LongTensor data type representing the label with the highest probability (Paszke et al. 2019).    